# Basi di Dati Mod. 2 - SQLAlchemy ORM

### Stefano Calzavara, Università Ca' Foscari Venezia

Gli ORM forniscono uno strumento per definire un mapping fra:
1. classi del linguaggio di programmazione (es. Python) e tabelle del DBMS sottostante
2. istanze di tali classi (oggetti) e righe delle tabelle corrispondenti

Il risultato è un sistema che sincronizza trasparentemente tutti i cambiamenti di stato fra gli oggetti e le righe ad essi associate. Tale approccio permette inoltre di esprimere query al database in termini di classi e delle relazioni esistenti fra di esse.

![alt text](orm.jpg "ORM architecture")

SQLAlchemy ORM è costruito sopra all'Expression Language ed offre un livello di astrazione ancora più elevato rispetto al DBMS sottostante. La maggior parte delle applicazioni può fare uso esclusivo delle funzionalità ORM, usando l'Expression Language solo dove è veramente necessario scendere a più basso livello.

### Dichiarazione di un mapping

In [2]:
import sqlalchemy
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base

engine = create_engine('sqlite://', echo=True)

Base = declarative_base()                      # tabella = classe che eredita da Base

class User(Base):
    __tablename__ = 'users'                   # obbligatorio

    id = Column(Integer, primary_key=True)    # almeno un attributo deve fare parte della primary key
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    # questo metodo è opzionale, serve solo per pretty printing
    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)

### Creare uno schema

In [3]:
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [4]:
Base.metadata.create_all(engine)

2022-04-09 15:10:23,706 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:10:23,708 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-04-09 15:10:23,709 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:10:23,711 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2022-04-09 15:10:23,711 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:10:23,712 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2022-04-09 15:10:23,713 INFO sqlalchemy.engine.Engine [no key 0.00054s] ()
2022-04-09 15:10:23,714 INFO sqlalchemy.engine.Engine COMMIT


### Creare un'istanza di una classe mappata

Un'istanza di una classe mappata rappresenta una riga della tabella corrispondente.

In [5]:
ed_user = User(name='ed', fullname='Ed Jones', nickname='thunder')
print(ed_user.name)
print(ed_user.nickname)
print(ed_user.id)

ed
thunder
None


Importante: Si noti che a questo punto non è ancora stato scritto niente nel database! Questo è confermato dal fatto che l'attributo `id` è ancora impostato a `None`. Abbiamo però preparato SQLAlchemy ORM affinché sia in grado di aggiornare il database correttamente all'interno di una **sessione**.

### Sessioni: creazione ed utilizzo

Una sessione in SQLAlchemy ORM nasconde una serie di dettagli implementativi che tipicamente sono gestiti manualmente nell'Expression Language, in particolare la gestione delle connessioni e delle transazioni.

In [6]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)       # factory pattern
session = Session()

In [7]:
session.add(ed_user)    # pending instance: verrà salvata nel database quando veramente necessario

In [8]:
our_user = session.query(User).filter_by(name='ed').first()    # qui è necessario salvare la pending instance
our_user

2022-04-09 15:10:47,889 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:10:47,892 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-09 15:10:47,893 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ('ed', 'Ed Jones', 'thunder')
2022-04-09 15:10:47,896 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2022-04-09 15:10:47,896 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='thunder')>

In [9]:
ed_user is our_user

True

In [10]:
print(ed_user.id)    # primary key creata in fase di scrittura al database

1


In [11]:
ed_user.nickname = 'eddie'

session.add_all([User(name='wendy', fullname='Wendy Williams', nickname='windy'),
                 User(name='mary', fullname='Mary Contrary', nickname='mary'),
                 User(name='fred', fullname='Fred Flintstone', nickname='freddy')])

In [12]:
print("Dirty instances: " + str(session.dirty))
print("Pending instances: " + str(session.new) + "\n")
session.commit()
print("\nDirty instances: " + str(session.dirty))
print("Pending instances: " + str(session.new))

Dirty instances: IdentitySet([<User(name='ed', fullname='Ed Jones', nickname='eddie')>])
Pending instances: IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

2022-04-09 15:10:55,389 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2022-04-09 15:10:55,391 INFO sqlalchemy.engine.Engine [generated in 0.00165s] ('eddie', 1)
2022-04-09 15:10:55,392 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-09 15:10:55,393 INFO sqlalchemy.engine.Engine [cached since 7.501s ago] ('wendy', 'Wendy Williams', 'windy')
2022-04-09 15:10:55,393 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-09 15:10:55,394 INFO sqlalchemy.engine.Engine [cached since 7.502s ago] ('mary', 'Mary Contrary', 'mary')
2022-04-09 15:10:55,394

In [13]:
ed_user.name = 'Edwardo'
fake_user = User(name='fakeuser', fullname='Invalid', nickname='12345')
session.add(fake_user)
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2022-04-09 15:10:56,037 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:10:56,039 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2022-04-09 15:10:56,040 INFO sqlalchemy.engine.Engine [generated in 0.00089s] (1,)
2022-04-09 15:10:56,042 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2022-04-09 15:10:56,042 INFO sqlalchemy.engine.Engine [generated in 0.00061s] ('Edwardo', 1)
2022-04-09 15:10:56,044 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-09 15:10:56,044 INFO sqlalchemy.engine.Engine [cached since 8.152s ago] ('fakeuser', 'Invalid', '12345')
2022-04-09 15:10:56,046 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2022-04-09 15:10:56,047 INFO 

[<User(name='Edwardo', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fakeuser', fullname='Invalid', nickname='12345')>]

In [14]:
session.rollback()
print(">>> eddie's name: " + ed_user.name)
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2022-04-09 15:10:56,951 INFO sqlalchemy.engine.Engine ROLLBACK
2022-04-09 15:10:56,953 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:10:56,954 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2022-04-09 15:10:56,955 INFO sqlalchemy.engine.Engine [generated in 0.00048s] (1,)
>>> eddie's name: ed
2022-04-09 15:10:56,956 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2022-04-09 15:10:56,957 INFO sqlalchemy.engine.Engine [cached since 0.9114s ago] ('ed', 'fakeuser')


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>]

### Selezionare dati

In [15]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2022-04-09 15:10:59,077 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2022-04-09 15:10:59,078 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [16]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2022-04-09 15:10:59,974 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2022-04-09 15:10:59,975 INFO sqlalchemy.engine.Engine [generated in 0.00116s] ()
ed Ed Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [17]:
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

2022-04-09 15:11:00,505 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2022-04-09 15:11:00,506 INFO sqlalchemy.engine.Engine [generated in 0.00140s] (2, 1)
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [18]:
for u in session.query(User).filter_by(fullname='Ed Jones'):
    print(u.name)

2022-04-09 15:11:01,130 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = ?
2022-04-09 15:11:01,131 INFO sqlalchemy.engine.Engine [generated in 0.00157s] ('Ed Jones',)
ed


In [19]:
for u in session.query(User).filter(User.fullname=='Ed Jones'):
    print(u.name)

2022-04-09 15:11:01,520 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.fullname = ?
2022-04-09 15:11:01,521 INFO sqlalchemy.engine.Engine [cached since 0.3917s ago] ('Ed Jones',)
ed


In [20]:
for u in session.query(User).filter(User.name.like('%ed')).filter(User.fullname=='Ed Jones'):
    print(u.name)

2022-04-09 15:11:01,928 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? AND users.fullname = ?
2022-04-09 15:11:01,930 INFO sqlalchemy.engine.Engine [generated in 0.00182s] ('%ed', 'Ed Jones')
ed


In [21]:
query = session.query(User).filter(User.name.like('%ed')).order_by(User.id)
query.all()

2022-04-09 15:11:02,155 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2022-04-09 15:11:02,158 INFO sqlalchemy.engine.Engine [generated in 0.00326s] ('%ed',)


[<User(name='ed', fullname='Ed Jones', nickname='eddie')>,
 <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>]

In [22]:
query.first()

2022-04-09 15:11:02,336 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
 LIMIT ? OFFSET ?
2022-04-09 15:11:02,339 INFO sqlalchemy.engine.Engine [generated in 0.00366s] ('%ed', 1, 0)


<User(name='ed', fullname='Ed Jones', nickname='eddie')>

In [23]:
query.one()

2022-04-09 15:11:03,064 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name LIKE ? ORDER BY users.id
2022-04-09 15:11:03,066 INFO sqlalchemy.engine.Engine [cached since 0.9107s ago] ('%ed',)


MultipleResultsFound: Multiple rows were found when exactly one was required

In [ ]:
query.count()

### Relazioni

Modelliamo ora una relazione uno-a-molti fra utenti ed indirizzi email. Tale relazione consente di effettuare le seguenti operazioni:
1. Dato un indirizzo email, trovare l'utente ad esso associato
2. Dato un utente, trovare una lista dei suoi indirizzi email

Questo meccanismo viene implementato tramite `relationship` come segue. Si faccia attenzione alla creazione di due attributi in questo caso, uno per ciascuna classe. Il primo attributo implementa 1, mentre il secondo attributo implementa 2.

In [24]:
from sqlalchemy.orm import relationship

class Address(Base):
     __tablename__ = 'addresses'
     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey(User.id))

     user = relationship(User, back_populates="addresses")    # qui viene sfruttata la foreign key

     def __repr__(self):
         return "<Address(email_address='%s')>" % self.email_address
        
User.addresses = relationship(Address, order_by=Address.id, back_populates="user")

In [25]:
Base.metadata.create_all(engine)

jack = User(name='jack', fullname='Jack Bean', nickname='j&b')
jack.addresses

2022-04-09 15:11:19,323 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:11:19,326 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-04-09 15:11:19,327 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:11:19,328 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-04-09 15:11:19,328 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:11:19,329 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("addresses")
2022-04-09 15:11:19,330 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:11:19,330 INFO sqlalchemy.engine.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2022-04-09 15:11:19,331 INFO sqlalchemy.engine.Engine [no key 0.00053s] ()
2022-04-09 15:11:19,332 INFO sqlalchemy.engine.Engine COMMIT


[]

In [26]:
jack.addresses = [Address(email_address='jack@google.com'), Address(email_address='j25@yahoo.com')]

In [27]:
jack.addresses[0]

<Address(email_address='jack@google.com')>

In [28]:
jack.addresses[0].user

<User(name='jack', fullname='Jack Bean', nickname='j&b')>

In [29]:
session.add(jack)
session.commit()

2022-04-09 15:12:01,885 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2022-04-09 15:12:01,888 INFO sqlalchemy.engine.Engine [generated in 0.00227s] ('jack', 'Jack Bean', 'j&b')
2022-04-09 15:12:01,890 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2022-04-09 15:12:01,891 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ('jack@google.com', 5)
2022-04-09 15:12:01,892 INFO sqlalchemy.engine.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2022-04-09 15:12:01,893 INFO sqlalchemy.engine.Engine [cached since 0.002685s ago] ('j25@yahoo.com', 5)
2022-04-09 15:12:01,894 INFO sqlalchemy.engine.Engine COMMIT


In [30]:
jack = session.query(User).filter_by(name='jack').one()
print(jack)     # nota: nessuna istruzione SQL viene eseguita per la tabella addresses

2022-04-09 15:12:04,002 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:12:04,008 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2022-04-09 15:12:04,009 INFO sqlalchemy.engine.Engine [generated in 0.00134s] ('jack',)
<User(name='jack', fullname='Jack Bean', nickname='j&b')>


In [31]:
jack.addresses   # solo a questo punto viene eseguito SQL per la tabella addresses

2022-04-09 15:12:05,151 INFO sqlalchemy.engine.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id ORDER BY addresses.id
2022-04-09 15:12:05,153 INFO sqlalchemy.engine.Engine [generated in 0.00143s] (5,)


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

### Giunzioni

In [32]:
for u, a in session.query(User, Address):
    print("({}, {})".format(u,a))

2022-04-09 15:12:09,540 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses
2022-04-09 15:12:09,542 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ()
(<User(name='ed', fullname='Ed Jones', nickname='eddie')>, <Address(email_address='jack@google.com')>)
(<User(name='ed', fullname='Ed Jones', nickname='eddie')>, <Address(email_address='j25@yahoo.com')>)
(<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <Address(email_address='jack@google.com')>)
(<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <Address(email_address='j25@yahoo.com')>)
(<User(name='mary', fullname='Mary Contrary', nickname='mary')>, <Address(email_address='jack@google.com')>)
(<User(name='mary', fullname='Mary Contrary', nickname='mary')>

/tmp/ipykernel_7399/691745299.py:1: SAWarning: SELECT statement has a cartesian product between FROM element(s) "addresses" and FROM element "users".  Apply join condition(s) between each element to resolve.
  for u, a in session.query(User, Address):


In [33]:
for u, a in session.query(User, Address).filter(User.id == Address.user_id):
    print("({}, {})".format(u,a))

2022-04-09 15:12:10,466 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM users, addresses 
WHERE users.id = addresses.user_id
2022-04-09 15:12:10,468 INFO sqlalchemy.engine.Engine [generated in 0.00220s] ()
(<User(name='jack', fullname='Jack Bean', nickname='j&b')>, <Address(email_address='jack@google.com')>)
(<User(name='jack', fullname='Jack Bean', nickname='j&b')>, <Address(email_address='j25@yahoo.com')>)


In [34]:
session.query(User).join(Address).count()

2022-04-09 15:12:11,456 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id) AS anon_1
2022-04-09 15:12:11,458 INFO sqlalchemy.engine.Engine [generated in 0.00151s] ()


2

In [35]:
# https://docs.sqlalchemy.org/en/13/faq/sessions.html#faq-query-deduplicating
session.query(User).join(Address).all()

2022-04-09 15:12:12,152 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id
2022-04-09 15:12:12,154 INFO sqlalchemy.engine.Engine [generated in 0.00179s] ()


[<User(name='jack', fullname='Jack Bean', nickname='j&b')>]

In [36]:
session.query(User.name, User.fullname, User.nickname).join(Address).all() 

2022-04-09 15:12:12,377 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users JOIN addresses ON users.id = addresses.user_id
2022-04-09 15:12:12,380 INFO sqlalchemy.engine.Engine [generated in 0.00287s] ()


[('jack', 'Jack Bean', 'j&b'), ('jack', 'Jack Bean', 'j&b')]

In [37]:
session.query(User, Address.email_address).join(Address).all()

2022-04-09 15:12:12,568 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.email_address AS addresses_email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2022-04-09 15:12:12,570 INFO sqlalchemy.engine.Engine [generated in 0.00189s] ()


[(<User(name='jack', fullname='Jack Bean', nickname='j&b')>, 'jack@google.com'),
 (<User(name='jack', fullname='Jack Bean', nickname='j&b')>, 'j25@yahoo.com')]

In [38]:
session.query(User, Address.email_address).outerjoin(User.addresses).all()

2022-04-09 15:12:12,788 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, addresses.email_address AS addresses_email_address 
FROM users LEFT OUTER JOIN addresses ON users.id = addresses.user_id
2022-04-09 15:12:12,790 INFO sqlalchemy.engine.Engine [generated in 0.00185s] ()


[(<User(name='ed', fullname='Ed Jones', nickname='eddie')>, None),
 (<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, None),
 (<User(name='mary', fullname='Mary Contrary', nickname='mary')>, None),
 (<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>, None),
 (<User(name='jack', fullname='Jack Bean', nickname='j&b')>, 'j25@yahoo.com'),
 (<User(name='jack', fullname='Jack Bean', nickname='j&b')>, 'jack@google.com')]

### Cancellazioni

In [39]:
session.delete(jack)
print("Deleted instances: " + str(session.deleted))
print(session.query(User).filter(User.name == 'jack').count())
print("Deleted instances: " + str(session.deleted))

Deleted instances: IdentitySet([<User(name='jack', fullname='Jack Bean', nickname='j&b')>])
2022-04-09 15:12:13,252 INFO sqlalchemy.engine.Engine UPDATE addresses SET user_id=? WHERE addresses.id = ?
2022-04-09 15:12:13,254 INFO sqlalchemy.engine.Engine [generated in 0.00118s] ((None, 1), (None, 2))
2022-04-09 15:12:13,255 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-04-09 15:12:13,256 INFO sqlalchemy.engine.Engine [generated in 0.00054s] (5,)
2022-04-09 15:12:13,258 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2022-04-09 15:12:13,259 INFO sqlalchemy.engine.Engine [generated in 0.00060s] ('jack',)
0
Deleted instances: IdentitySet([])


In [40]:
session.query(Address).all()    # nessuna forma di cascading!

2022-04-09 15:12:14,227 INFO sqlalchemy.engine.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses
2022-04-09 15:12:14,228 INFO sqlalchemy.engine.Engine [generated in 0.00150s] ()


[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]

In [41]:
session.rollback()

Base = declarative_base()

class User(Base):
     __tablename__ = 'users'                   # obbligatorio

     id = Column(Integer, primary_key=True)    # almeno un attributo deve fare parte della primary key
     name = Column(String)
     fullname = Column(String)
     nickname = Column(String)
     
     # configuriamo la politica di cascading
     addresses = relationship("Address", back_populates='user', cascade="all, delete, delete-orphan")

     def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)
    
class Address(Base):
     __tablename__ = 'addresses'
     id = Column(Integer, primary_key=True)
     email_address = Column(String, nullable=False)
     user_id = Column(Integer, ForeignKey(User.id))

     user = relationship(User, back_populates="addresses")

     def __repr__(self):
         return "<Address(email_address='%s')>" % self.email_address

2022-04-09 15:12:14,417 INFO sqlalchemy.engine.Engine ROLLBACK


In [42]:
jack = session.query(User).filter(User.name == 'jack').first()

for a in jack.addresses:
    print(a)

2022-04-09 15:12:15,436 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:12:15,437 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2022-04-09 15:12:15,438 INFO sqlalchemy.engine.Engine [generated in 0.00084s] ('jack', 1, 0)
2022-04-09 15:12:15,441 INFO sqlalchemy.engine.Engine SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE ? = addresses.user_id
2022-04-09 15:12:15,442 INFO sqlalchemy.engine.Engine [generated in 0.00056s] (5,)
<Address(email_address='jack@google.com')>
<Address(email_address='j25@yahoo.com')>


In [43]:
del jack.addresses[1]
session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count()

2022-04-09 15:12:16,100 INFO sqlalchemy.engine.Engine DELETE FROM addresses WHERE addresses.id = ?
2022-04-09 15:12:16,102 INFO sqlalchemy.engine.Engine [generated in 0.00168s] (2,)
2022-04-09 15:12:16,105 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2022-04-09 15:12:16,106 INFO sqlalchemy.engine.Engine [generated in 0.00066s] ('jack@google.com', 'j25@yahoo.com')


1

In [44]:
session.delete(jack)
print(session.query(User).filter(User.name == 'jack').count())
print(session.query(Address).filter(Address.email_address.in_(['jack@google.com', 'j25@yahoo.com'])).count())

2022-04-09 15:12:17,116 INFO sqlalchemy.engine.Engine DELETE FROM addresses WHERE addresses.id = ?
2022-04-09 15:12:17,117 INFO sqlalchemy.engine.Engine [cached since 1.017s ago] (1,)
2022-04-09 15:12:17,118 INFO sqlalchemy.engine.Engine DELETE FROM users WHERE users.id = ?
2022-04-09 15:12:17,119 INFO sqlalchemy.engine.Engine [generated in 0.00045s] (5,)
2022-04-09 15:12:17,121 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?) AS anon_1
2022-04-09 15:12:17,122 INFO sqlalchemy.engine.Engine [generated in 0.00099s] ('jack',)
0
2022-04-09 15:12:17,124 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT addresses.id AS addresses_id, addresses.email_address AS addresses_email_address, addresses.user_id AS addresses_user_id 
FROM addresses 
WHERE addresses.email_address IN (?, ?)) AS anon_1
2022-04-09 15

### Relazioni molti-a-molti

Modelliamo infine una relazione molti-a-molti fra blog post e keywords al loro interno. Ciò richiede la creazione di una **tabella di associazione**, che sporca l'eleganza della rappresentazione ORM vista fino ad ora. Tale tabella deve avere esattamente due colonne, che operano da chiavi esterne verso le due relazioni da associare.

Tramite la tabella di associazione è possibile effettuare le seguenti operazioni:
1. Dato un blog post, trovare la lista delle sue keywords
2. Data una keyword, trovare la lista dei blog post in cui occorre

Sebbene siamo costretti ad esporre tale dettaglio implementativo, l'interfaccia di accesso alle informazioni desiderate tramite l'ORM rimane comunque estremamente semplice. E' possibile estendere tale pattern a strutture con più di due colonne usando **oggetti di associazione**: per i dettagli potete consultare la documentazione ufficiale [qui](https://docs.sqlalchemy.org/en/13/orm/basic_relationships.html#relationship-patterns).

In [45]:
from sqlalchemy import Table

post_keywords = Table('post_keywords', Base.metadata,
                                       Column('post_id', ForeignKey('posts.id'), primary_key=True),
                                       Column('keyword_id', ForeignKey('keywords.id'), primary_key=True))

In [46]:
class BlogPost(Base):
    __tablename__ = 'posts'

    id = Column(Integer, primary_key=True)
    user_id = Column(Integer, ForeignKey('users.id'))
    headline = Column(String(255), nullable=False)
    body = Column(Text)

    # relazione molti-a-molti
    keywords = relationship('Keyword', secondary=post_keywords, back_populates='posts')

    def __repr__(self):
        return "BlogPost(%r, %r, %r)" % (self.headline, self.body, self.author)
    
class Keyword(Base):
    __tablename__ = 'keywords'

    id = Column(Integer, primary_key=True)
    kw = Column(String(50), nullable=False, unique=True)
    
    # relazione molti-a-molti
    posts = relationship('BlogPost', secondary=post_keywords, back_populates='keywords')
    
    # costruttore esplicito (opzionale)
    def __init__(self, kw):
        self.kw = kw

Aggiungiamo poi una relazione uno-a-molti fra utenti e blog post, riutilizzando le tecniche già viste. Si noti che avevamo già introdotto una chiave esterna su `BlogPost` per questo compito.

In [47]:
BlogPost.author = relationship(User, back_populates="posts")
User.posts = relationship(BlogPost, back_populates="author")

In [48]:
Base.metadata.create_all(engine)

2022-04-09 15:12:20,832 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-04-09 15:12:20,835 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2022-04-09 15:12:20,837 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:12:20,839 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("addresses")
2022-04-09 15:12:20,841 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:12:20,842 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("post_keywords")
2022-04-09 15:12:20,845 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:12:20,846 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("post_keywords")
2022-04-09 15:12:20,847 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:12:20,849 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("posts")
2022-04-09 15:12:20,850 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-09 15:12:20,851 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("posts")
2022-04-09 15:12:20,852 INFO sqlalchemy.engine.Engine [raw

In [49]:
wendy = session.query(User).filter_by(name='wendy').one()
post = BlogPost(headline="Wendy's Blog Post", body="This is a test", author=wendy)
session.add(post)

2022-04-09 15:12:21,025 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2022-04-09 15:12:21,028 INFO sqlalchemy.engine.Engine [generated in 0.00300s] ('wendy',)


In [50]:
post.keywords = [Keyword('wendy'), Keyword('firstpost')]

In [51]:
session.query(BlogPost).filter(BlogPost.keywords.any(kw='firstpost')).all()

2022-04-09 15:12:21,387 INFO sqlalchemy.engine.Engine INSERT INTO keywords (kw) VALUES (?)
2022-04-09 15:12:21,390 INFO sqlalchemy.engine.Engine [generated in 0.00259s] ('wendy',)
2022-04-09 15:12:21,393 INFO sqlalchemy.engine.Engine INSERT INTO keywords (kw) VALUES (?)
2022-04-09 15:12:21,394 INFO sqlalchemy.engine.Engine [cached since 0.007004s ago] ('firstpost',)
2022-04-09 15:12:21,397 INFO sqlalchemy.engine.Engine INSERT INTO posts (user_id, headline, body) VALUES (?, ?, ?)
2022-04-09 15:12:21,398 INFO sqlalchemy.engine.Engine [generated in 0.00118s] (2, "Wendy's Blog Post", 'This is a test')
2022-04-09 15:12:21,400 INFO sqlalchemy.engine.Engine INSERT INTO post_keywords (post_id, keyword_id) VALUES (?, ?)
2022-04-09 15:12:21,401 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ((1, 1), (1, 2))
2022-04-09 15:12:21,405 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM post

[BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)]

In [52]:
session.query(BlogPost).filter(BlogPost.author == wendy).all()

2022-04-09 15:12:21,590 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE ? = posts.user_id
2022-04-09 15:12:21,593 INFO sqlalchemy.engine.Engine [generated in 0.00284s] (2,)


[BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)]

In [53]:
wendy.posts

2022-04-09 15:12:21,762 INFO sqlalchemy.engine.Engine SELECT posts.id AS posts_id, posts.user_id AS posts_user_id, posts.headline AS posts_headline, posts.body AS posts_body 
FROM posts 
WHERE ? = posts.user_id
2022-04-09 15:12:21,765 INFO sqlalchemy.engine.Engine [generated in 0.00312s] (2,)


[BlogPost("Wendy's Blog Post", 'This is a test', <User(name='wendy', fullname='Wendy Williams', nickname='windy')>)]

## Esercizio

Creare tramite SQLAlchemy ORM le seguenti tabelle:
* Product(maker, model*, type)
* PC(<u>model*</u>, speed, ram, hd, price)
* Laptop(<u>model*</u>, speed, ram, hd, screen, price)
* Printer(<u>model*</u>, color, type, price)

Definite opportune chiavi primarie ed esterne, scegliendo i tipi di dato appropriati per i vari attributi. Una volta fatto ciò, popolate le tabelle con alcuni dati artificiali ed effettuate tramite l'ORM di SQLAlchemy le seguenti query:
1. Trovare il modello, la velocità e la dimensione dell'hard disk di tutti i PC che costano meno di $1000. 
2. Trovare tutti i produttori di stampanti.
3. Trovare il produttore e la velocità dei laptop con un hard disk da almeno 300 GB.
4. Trovare il modello ed il prezzo di tutti i PC ed i laptop realizzati dalla Lenovo.
5. Trovare le dimensioni degli hard disk che occorrono in almeno due PC.
6. Trovare tutte le aziende che producono laptop, ma non PC.
7. Trovare i produttori di PC con una velocità minima di 2.0 GHz.
8. Trovare tutte le aziende che producono sia PC che laptop.

Procedete immaginando la query SQL e traducendola nell'ORM di SQLAlchemy. Consultate la documentazione dove necessario.